In [1]:
%pip install pandas numpy matplotlib seaborn scipy statsmodels

import warnings
warnings.filterwarnings("ignore")

import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multicomp import pairwise_tukeyhsd

sns.set(style="whitegrid")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scipy-1.16.3-cp314-cp314-win_amd64.whl.metadata (60 kB)
  Using cached patsy-1.0.2-py2.py3-none-any.whl.metadata (3.6 kB)
Using cached scipy-1.16.3-cp314-cp314-win_amd64.whl (39.4 MB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.6 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/9.6 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/9.6 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/9.6 MB 642.2 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/9.6 MB 642.2 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/9.6 MB 642.2 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/9.6 MB 642.2 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/9.6 MB

In [2]:
# -------------------------
# Helper functions
# -------------------------
def safe_read(path: str):
    return pd.read_csv(path, sep="|", parse_dates=["TransactionMonth","VehicleIntroDate"], low_memory=False)

def ensure_columns(df: pd.DataFrame):
    # Ensure required columns exist (create if missing)
    if "TotalPremium" not in df.columns:
        df["TotalPremium"] = np.nan
    if "TotalClaims" not in df.columns:
        df["TotalClaims"] = 0.0
    if "PostalCode" not in df.columns and "PostalCode" in df.columns:
        pass
    # Some datasets use PostalCode or Postalcode or Postal_Code - try common alternatives
    if "PostalCode" not in df.columns:
        for alt in ["Postalcode","postalcode","ZIP","ZipCode","Postal_Code","Postal"]:
            if alt in df.columns:
                df["PostalCode"] = df[alt]
                break
    if "Province" not in df.columns:
        df["Province"] = df.get("province", np.nan)
    if "Gender" not in df.columns:
        df["Gender"] = df.get("gender", np.nan)
    return df

def create_metrics(df: pd.DataFrame):
    # Claim indicator (has at least one claim)
    df["HasClaim"] = (df["TotalClaims"].fillna(0) > 0).astype(int)
    # Claim frequency per group = mean(HasClaim)
    # Claim severity: average claim size *given a claim occurred*
    df["ClaimSeverity"] = df.apply(lambda r: r["TotalClaims"] if r["HasClaim"]==1 else np.nan, axis=1)
    # Margin
    df["Margin"] = df["TotalPremium"].fillna(0) - df["TotalClaims"].fillna(0)
    return df

def summary_by_group(df, group_col, metrics=["HasClaim","ClaimSeverity","Margin"]):
    grp = df.groupby(group_col).agg(
        policies=('HasClaim','count'),
        claim_freq=('HasClaim','mean'),
        claim_freq_count=('HasClaim','sum'),
        claim_severity_mean=('ClaimSeverity','mean'),
        claim_severity_median=('ClaimSeverity','median'),
        margin_mean=('Margin','mean'),
        margin_median=('Margin','median')
    ).reset_index()
    return grp

def chi2_test_contingency(df, cat_col, outcome_col="HasClaim"):
    # Create contingency table outcome x category
    table = pd.crosstab(df[cat_col], df[outcome_col])
    chi2, p, dof, expected = stats.chi2_contingency(table)
    return {"chi2":chi2, "p":p, "dof":dof, "expected":expected, "table":table}

def test_proportion_z(group1_count, group1_n, group2_count, group2_n):
    counts = np.array([group1_count, group2_count])
    nobs = np.array([group1_n, group2_n])
    stat, pval = proportions_ztest(count=counts, nobs=nobs, alternative="two-sided")
    return stat, pval

def compare_continuous_ttest(group_a, group_b, alternative="two-sided"):
    # Welch's t-test (unequal variances)
    stat, p = stats.ttest_ind(group_a.dropna(), group_b.dropna(), equal_var=False, nan_policy='omit')
    return stat, p

def compare_continuous_mwu(group_a, group_b):
    stat, p = stats.mannwhitneyu(group_a.dropna(), group_b.dropna(), alternative="two-sided")
    return stat, p

def ks_test(group_a, group_b):
    stat, p = stats.ks_2samp(group_a.dropna(), group_b.dropna())
    return stat, p

In [3]:
# -------------------------
# Main execution
# -------------------------
if __name__ == "__main__":
    # Paths
    data_path = Path("../data/MachineLearningRating_v3.txt")
    if not data_path.exists():
        print(f"ERROR: Data file not found at {data_path.resolve()}")
        print("Please put MachineLearningRating_v3.txt into data")
        raise SystemExit

    print("Loading data...")
    df = safe_read(str(data_path))
    df = ensure_columns(df)
    print("Creating metrics...")
    df = create_metrics(df)

    # Basic counts
    n_total = len(df)
    n_with_claims = df["HasClaim"].sum()
    overall_freq = df["HasClaim"].mean()
    overall_severity = df["ClaimSeverity"].mean()
    overall_margin = df["Margin"].mean()

    print(f"\nDataset loaded: {n_total} policies")
    print(f"Overall claim frequency: {overall_freq:.4f} ({int(n_with_claims)}/{n_total})")
    print(f"Overall claim severity (conditional mean): {overall_severity:.2f}")
    print(f"Overall margin (mean): {overall_margin:.2f}\n")


Loading data...
Creating metrics...

Dataset loaded: 1000098 policies
Overall claim frequency: 0.0028 (2788/1000098)
Overall claim severity (conditional mean): 23273.39
Overall margin (mean): -2.96



In [5]:
# -------------------------
    # H0: No risk differences across provinces (Claim Frequency)
    # Use chi-square on HasClaim x Province
    # -------------------------
print("TEST 1 — Risk differences across provinces (Claim Frequency)")
if df["Province"].isna().all():
        print("No Province information available — cannot test province-level risk.")
        province_result = None
else:
        chi2_res = chi2_test_contingency(df.dropna(subset=["Province"]), "Province", "HasClaim")
        p = chi2_res["p"]
        print(f"Chi-square: {chi2_res['chi2']:.3f}, p-value = {p:.6f}, dof = {chi2_res['dof']}")
        if p < 0.05:
            print("RESULT: Reject H0 — there are risk differences across provinces (p < 0.05).")
        else:
            print("RESULT: Fail to reject H0 — no evidence of province-level differences (p >= 0.05).")
        # Save some summary
        province_summary = summary_by_group(df.dropna(subset=["Province"]), "Province")
        province_result = {"chi2":chi2_res, "summary": province_summary}

    # Also check severity differences across provinces (Kruskal-Wallis)
print("\nTEST 1b — Claim Severity differences across provinces (non-parametric)")
try:
        # keep provinces with reasonable sample of claims
        prov_claims = df.dropna(subset=["Province"])
        groups = [g["ClaimSeverity"].dropna() for _, g in prov_claims.groupby("Province") if g["HasClaim"].sum() >= 10]
        if len(groups) >= 2:
            kw_stat, kw_p = stats.kruskal(*groups)
            print(f"Kruskal-Wallis stat: {kw_stat:.3f}, p-value = {kw_p:.6f}")
            if kw_p < 0.05:
                print("RESULT: Reject H0 — claim severity differs across provinces.")
            else:
                print("RESULT: Fail to reject H0 — no evidence of severity differences across provinces.")
        else:
            print("Not enough provinces with 10+ claims to run Kruskal-Wallis reliably.")
except Exception as e:
        print("Severity test failed:", e)

TEST 1 — Risk differences across provinces (Claim Frequency)
Chi-square: 104.191, p-value = 0.000000, dof = 8
RESULT: Reject H0 — there are risk differences across provinces (p < 0.05).

TEST 1b — Claim Severity differences across provinces (non-parametric)
Kruskal-Wallis stat: 105.754, p-value = 0.000000
RESULT: Reject H0 — claim severity differs across provinces.


In [6]:
# -------------------------
    # H0: No risk differences between zip codes (PostalCode)
    # We'll restrict to the top-k PostalCodes by sample size (to avoid tiny counts)
    # -------------------------
print("\nTEST 2 — Risk differences between zip codes (Claim Frequency)")
if df["PostalCode"].isna().all():
        print("No PostalCode available — cannot test zip code differences.")
        zip_result = None
else:
        top_k = 10
        top_zips = df["PostalCode"].value_counts().head(top_k).index.tolist()
        df_topz = df[df["PostalCode"].isin(top_zips)].copy()
        print(f"Testing top {len(top_zips)} PostalCodes by frequency: {top_zips}")
        chi2_res_zip = chi2_test_contingency(df_topz, "PostalCode", "HasClaim")
        pzip = chi2_res_zip["p"]
        print(f"Chi-square: {chi2_res_zip['chi2']:.3f}, p-value = {pzip:.6f}, dof = {chi2_res_zip['dof']}")
        if pzip < 0.05:
            print("RESULT: Reject H0 — risk differs across these zip codes (top 10).")
        else:
            print("RESULT: Fail to reject H0 — no evidence of zip-level frequency differences among top zips.")
        zip_result = {"chi2": chi2_res_zip, "top_zips": top_zips, "summary": summary_by_group(df_topz, "PostalCode")}


TEST 2 — Risk differences between zip codes (Claim Frequency)
Testing top 10 PostalCodes by frequency: [2000, 122, 7784, 299, 7405, 458, 8000, 2196, 470, 7100]
Chi-square: 72.649, p-value = 0.000000, dof = 9
RESULT: Reject H0 — risk differs across these zip codes (top 10).


In [7]:
# -------------------------
    # H0: There is no significant margin (profit) difference between zip codes
    # Use ANOVA or Kruskal-Wallis on Margin across top postal codes
    # -------------------------
print("\nTEST 3 — Margin differences between zip codes")
if df["PostalCode"].isna().all():
        print("No PostalCode available — cannot test margin differences.")
        margin_zip_result = None
else:
        # Consider top_k chosen above
        margin_groups = [g["Margin"].dropna() for _, g in df_topz.groupby("PostalCode") if len(g) >= 30]
        if len(margin_groups) >= 2:
            # Use Kruskal-Wallis for robustness when not normal
            kw_stat_m, kw_p_m = stats.kruskal(*margin_groups)
            print(f"Kruskal-Wallis stat: {kw_stat_m:.3f}, p-value = {kw_p_m:.6f}")
            if kw_p_m < 0.05:
                print("RESULT: Reject H0 — Margin differs across these zip codes.")
            else:
                print("RESULT: Fail to reject H0 — no strong evidence of margin differences across these zip codes.")
            margin_zip_result = {"kw_stat": kw_stat_m, "p": kw_p_m}
        else:
            print("Not enough data per postal code (>=30) to run margin comparison reliably.")
            margin_zip_result = None

    # -------------------------
    # H0: There is no significant risk difference between Women and Men
    # Tests: Claim Frequency (chi-square), Claim Severity (t-test or Mann-Whitney)
    # -------------------------
print("\nTEST 4 — Risk differences between Women and Men")
if df["Gender"].isna().all():
        print("No Gender information available — cannot test gender differences.")
        gender_result = None
else:
        # Normalize gender labels (attempt)
        df["Gender_clean"] = df["Gender"].astype(str).str.strip().str.upper().replace({"FEMALE":"F","MALE":"M","UNK":"UNKNOWN"})
        gender_counts = df["Gender_clean"].value_counts()
        print("Gender counts:\n", gender_counts.head())
        # Consider just M and F groups if present
        if ("M" in gender_counts.index) and ("F" in gender_counts.index):
            group_m = df[df["Gender_clean"]=="M"]
            group_f = df[df["Gender_clean"]=="F"]

            # Frequency comparison (z-test)
            m_count = group_m["HasClaim"].sum()
            m_n = len(group_m)
            f_count = group_f["HasClaim"].sum()
            f_n = len(group_f)
            stat_z, p_z = test_proportion_z(m_count, m_n, f_count, f_n)
            print(f"Proportion z-test stat: {stat_z:.3f}, p-value = {p_z:.6f}")
            if p_z < 0.05:
                print("RESULT: Reject H0 — claim frequency differs between genders.")
            else:
                print("RESULT: Fail to reject H0 — no evidence of frequency difference by gender.")

            # Severity comparison (check distribution skew)
            m_sev = group_m["ClaimSeverity"].dropna()
            f_sev = group_f["ClaimSeverity"].dropna()
            if len(m_sev)>=20 and len(f_sev)>=20:
                # check normality quickly
                _, p_mn = stats.shapiro(m_sev.sample(min(5000,len(m_sev)))) if len(m_sev) < 5000 else (None, None)
                _, p_fn = stats.shapiro(f_sev.sample(min(5000,len(f_sev)))) if len(f_sev) < 5000 else (None, None)
                # heavy-tailed data -> use Mann-Whitney
                m_stat, m_p = compare_continuous_mwu(m_sev, f_sev)
                test_used = "Mann-Whitney U"
                print(f"{test_used} stat: {m_stat:.3f}, p-value = {m_p:.6f}")
                if m_p < 0.05:
                    print("RESULT: Reject H0 — claim severity differs between genders.")
                else:
                    print("RESULT: Fail to reject H0 — no evidence of severity difference by gender.")
                gender_result = {"freq_test":(stat_z,p_z), "severity_test":(m_stat,m_p), "test_used": test_used}
            else:
                print("Not enough claim severity data for both genders to compare robustly.")
                gender_result = {"freq_test":(stat_z,p_z), "severity_test":None}
        else:
            print("Not enough M/F labels to perform gender comparison cleanly.")
            gender_result = None


TEST 3 — Margin differences between zip codes
Kruskal-Wallis stat: 4931.140, p-value = 0.000000
RESULT: Reject H0 — Margin differs across these zip codes.

TEST 4 — Risk differences between Women and Men
Gender counts:
 Gender_clean
NOT SPECIFIED    940990
M                 42817
NAN                9536
F                  6755
Name: count, dtype: int64
Proportion z-test stat: 0.201, p-value = 0.840494
RESULT: Fail to reject H0 — no evidence of frequency difference by gender.
Not enough claim severity data for both genders to compare robustly.


In [8]:
 # -------------------------
    # Group balance checks (for A/B fairness)
    # For an example: if we choose two provinces or two postal codes as groups,
    # ensure distributions of other covariates are similar (simple checks shown).
    # -------------------------
print("\nGROUP BALANCE DIAGNOSTICS (example: top 2 provinces if available)")
try:
        prov_counts = df["Province"].value_counts().head(2)
        if len(prov_counts) >= 2:
            p1, p2 = prov_counts.index[:2]
            g1 = df[df["Province"]==p1]
            g2 = df[df["Province"]==p2]
            print(f"Comparing provinces: {p1} (n={len(g1)}), {p2} (n={len(g2)})")
            # Numeric checks: TotalPremium distribution (KS test)
            kp_stat, kp_p = ks_test(g1["TotalPremium"].dropna(), g2["TotalPremium"].dropna())
            print(f"KS test on TotalPremium: stat={kp_stat:.3f}, p={kp_p:.6f}")
            # Categorical checks: VehicleType (chi-square)
            if "VehicleType" in df.columns:
                ct = pd.crosstab(df["Province"], df["VehicleType"])
                chi2_v, p_v, _, _ = stats.chi2_contingency(ct.loc[[p1,p2]])
                print(f"VehicleType chi2 among the two provinces: chi2={chi2_v:.3f}, p={p_v:.6f}")
            else:
                print("VehicleType not present; skipping VehicleType balance check.")
            print("If p-values are small (<0.05), the two groups differ significantly on that covariate.")
        else:
            print("Not enough provinces to run balance diagnostics.")
except Exception as e:
        print("Group balance diagnostics failed:", e)


GROUP BALANCE DIAGNOSTICS (example: top 2 provinces if available)
Comparing provinces: Gauteng (n=393865), Western Cape (n=170796)
KS test on TotalPremium: stat=0.051, p=0.000000
VehicleType chi2 among the two provinces: chi2=3319.698, p=0.000000
If p-values are small (<0.05), the two groups differ significantly on that covariate.


In [9]:
# -------------------------
# Save summaries and outputs
# -------------------------
out_dir = Path("reports")
out_dir.mkdir(exist_ok=True)

# province summary
if 'province_summary' in locals():
    province_summary.to_csv(out_dir / "province_summary_task3.csv", index=False)
if 'province_result' in locals() and province_result is not None:
    province_result["summary"].to_csv(out_dir / "province_summary_full.csv", index=False)
if 'zip_result' in locals() and zip_result is not None:
    zip_result["summary"].to_csv(out_dir / "zip_summary_top10.csv", index=False)

# Produce a simple textual report
report_lines = []
report_lines.append("TASK 3: Hypothesis Testing Report")
report_lines.append(f"Total policies analyzed: {n_total}")
report_lines.append(f"Overall claim frequency: {overall_freq:.4f}")
report_lines.append("")

# Province interpretation
if province_result is None:
    report_lines.append("Province test: Not performed (no data).")
else:
    p_val = province_result["chi2"]["p"]
    if p_val < 0.05:
        # Find top provinces by loss ratio difference for business interpretation
        prov_df = province_result["summary"].copy()
        prov_df["loss_ratio"] = prov_df["claim_freq"]  # equal here
        prov_sorted = prov_df.sort_values("loss_ratio", ascending=False).head(5)
        lines = [f"Province test: Reject H0 (p={p_val:.4g}). Example high-risk provinces (by claim frequency):"]
        for _, r in prov_sorted.iterrows():
            lines.append(f"  - {r['Province']}: claim_freq={r['claim_freq']:.3f} (n={int(r['policies'])})")
        report_lines.extend(lines)
    else:
        report_lines.append(f"Province test: Fail to reject H0 (p={p_val:.4g}). No strong province-level frequency differences found.")

# Zip code interpretation
if 'zip_result' not in locals() or zip_result is None:
    report_lines.append("Zip code frequency test: Not performed (no data).")
else:
    pz = zip_result["chi2"]["p"]
    if pz < 0.05:
        report_lines.append(f"Zip code frequency test: Reject H0 (p={pz:.4g}). Top zip codes show differing claim rates — consider postcode-based segmentation for top postal areas.")
    else:
        report_lines.append(f"Zip code frequency test: Fail to reject H0 (p={pz:.4g}). No strong evidence among top postal codes.")

# Margin zip interpretation
if margin_zip_result is None:
    report_lines.append("Zip code margin test: Not performed or insufficient data.")
else:
    pm = margin_zip_result.get("p", None)
    if pm is None:
        report_lines.append("Zip code margin test: Result available but p-value missing.")
    else:
        if pm < 0.05:
            report_lines.append(f"Zip code margin test: Reject H0 (p={pm:.4g}). Some postal codes show different profitability — consider underwriting/premium adjustments.")
        else:
            report_lines.append(f"Zip code margin test: Fail to reject H0 (p={pm:.4g}). No clear margin differences among tested postal codes.")

# Gender interpretation
if gender_result is None:
    report_lines.append("Gender tests: Not performed (insufficient or missing gender data).")
else:
    p_gender_freq = gender_result["freq_test"][1]
    report_lines.append(f"Gender frequency test p-value: {p_gender_freq:.4g}")
    if p_gender_freq < 0.05:
        report_lines.append("  - Interpretation: Reject H0. Differences in claim frequency exist between genders. Consider further analysis to identify behavioral drivers (note regulatory constraints before pricing on gender).")
    else:
        report_lines.append("  - Interpretation: Fail to reject H0. No strong evidence for frequency difference by gender.")

    if gender_result.get("severity_test") is not None:
        p_gender_sev = gender_result["severity_test"][1]
        if p_gender_sev < 0.05:
            report_lines.append(f"  - Severity test: Reject H0 (p={p_gender_sev:.4g}). Severity differs by gender.")
        else:
            report_lines.append(f"  - Severity test: Fail to reject H0 (p={p_gender_sev:.4g}). No evidence of severity difference by gender.")

# Write report to file
report_file = out_dir / "task3_hypothesis_report.txt"
with open(report_file, "w") as f:
    f.write("\n".join(report_lines))

print("\n" + "="*40)
print("SUMMARY (brief):")
for line in report_lines:
    print(line)
print("\nFull textual report saved to:", report_file.resolve())
print("CSV summaries (province/zip) saved to:", out_dir.resolve())
print("Task 3 EDA & tests completed. ✨")


SUMMARY (brief):
TASK 3: Hypothesis Testing Report
Total policies analyzed: 1000098
Overall claim frequency: 0.0028

Province test: Reject H0 (p=5.926e-19). Example high-risk provinces (by claim frequency):
  - Gauteng: claim_freq=0.003 (n=393865)
  - KwaZulu-Natal: claim_freq=0.003 (n=169781)
  - Limpopo: claim_freq=0.003 (n=24836)
  - North West: claim_freq=0.002 (n=143287)
  - Mpumalanga: claim_freq=0.002 (n=52718)
Zip code frequency test: Reject H0 (p=4.593e-12). Top zip codes show differing claim rates — consider postcode-based segmentation for top postal areas.
Zip code margin test: Reject H0 (p=0). Some postal codes show different profitability — consider underwriting/premium adjustments.
Gender frequency test p-value: 0.8405
  - Interpretation: Fail to reject H0. No strong evidence for frequency difference by gender.

Full textual report saved to: C:\Users\user\Desktop\Week-3\notebooks\reports\task3_hypothesis_report.txt
CSV summaries (province/zip) saved to: C:\Users\user\Des